# Development of Flight Map

In [1]:
import numpy as np
import pandas as pd

In [2]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [3]:
flights = pd.read_csv('data/flights.csv', index_col=0)
df = flights.copy()

In [4]:
flights = flights[['arr_delay', 'origin', 'dest', 'air_time', 'distance', 'carrier']]
flights['air_speed (mph)'] = flights['distance'] / (flights['air_time'] / 60)
flights.head()

,arr_delay,origin,dest,air_time,distance,carrier,air_speed (mph)
0,11.0,EWR,IAH,227.0,1400,UA,370.044053
1,20.0,LGA,IAH,227.0,1416,UA,374.273128
2,33.0,JFK,MIA,160.0,1089,AA,408.375000
3,-18.0,JFK,BQN,183.0,1576,B6,516.721311
4,-25.0,LGA,ATL,116.0,762,DL,394.137931


In [5]:
flights_grouped = flights.groupby(['origin', 'dest', 'carrier']).agg(['count', 'mean', 'min', 'max']).reset_index()
flights_grouped.head()

origin dest carrier arr_delay                         air_time              \
                          count       mean   min    max    count        mean   
0    EWR  ALB      EV       418  14.397129 -34.0  328.0      418   31.787081   
1    EWR  ANC      UA         8  -2.500000 -47.0   39.0        8  413.125000   
2    EWR  ATL      9E         4  -6.250000 -24.0   15.0        4  111.000000   
3    EWR  ATL      DL      3116   9.996470 -38.0  796.0     3116  110.853659   
4    EWR  ATL      EV      1654  19.546554 -39.0  454.0     1654  114.042322   

                distance                     air_speed (mph)              \
     min    max    count    mean   min   max           count        mean   
0   24.0   50.0      439   143.0   143   143             418  272.300976   
1  388.0  434.0        8  3370.0  3370  3370               8  489.985945   
2  105.0  117.0        4   746.0   746   746               4  403.899553   
3   88.0  176.0     3153   746.0   746   746            3116  406.542093   
4   90.0  161.0     1762   746.0   746   746            1654  395.945542   

                           
          min         max  
0  171.600000  357.500000  
1  465.898618  521.134021  
2  382.564103  426.285714  
3  254.318182  508.636364  
4  278.012422  497.333333

In [6]:
airlines = pd.read_csv('data/airlines.csv')
airlines_dict = {code: name for code, name in zip(airlines['carrier'], airlines['name'])}
flights_grouped['carrier'] = flights_grouped['carrier'].replace(airlines_dict)
flights_grouped.head()

origin dest                   carrier arr_delay                          \
                                            count       mean   min    max   
0    EWR  ALB  ExpressJet Airlines Inc.       418  14.397129 -34.0  328.0   
1    EWR  ANC     United Air Lines Inc.         8  -2.500000 -47.0   39.0   
2    EWR  ATL         Endeavor Air Inc.         4  -6.250000 -24.0   15.0   
3    EWR  ATL      Delta Air Lines Inc.      3116   9.996470 -38.0  796.0   
4    EWR  ATL  ExpressJet Airlines Inc.      1654  19.546554 -39.0  454.0   

  air_time                           distance                      \
     count        mean    min    max    count    mean   min   max   
0      418   31.787081   24.0   50.0      439   143.0   143   143   
1        8  413.125000  388.0  434.0        8  3370.0  3370  3370   
2        4  111.000000  105.0  117.0        4   746.0   746   746   
3     3116  110.853659   88.0  176.0     3153   746.0   746   746   
4     1654  114.042322   90.0  161.0     1762   746.0   746   746   

  air_speed (mph)                                      
            count        mean         min         max  
0             418  272.300976  171.600000  357.500000  
1               8  489.985945  465.898618  521.134021  
2               4  403.899553  382.564103  426.285714  
3            3116  406.542093  254.318182  508.636364  
4            1654  395.945542  278.012422  497.333333

In [7]:
set(airports['TZ'])

NameError: name 'airports' is not defined

In [8]:
from bokeh.sampledata.airport_routes import airports
airports = airports[airports['IATA'].isin(list(set(flights_grouped['origin'])) + list(set(flights_grouped['dest'])))]
airports = airports[~airports['TZ'].isin(['America/Anchorage', 'Pacific/Honolulu'])]
airports.head()

RuntimeError: bokeh sample data directory does not exist, please execute bokeh.sampledata.download()

In [86]:
airports_long = {code: longitude for code, longitude in zip(airports['IATA'], airports['Longitude'])}
airports_lati = {code: latitude for code, latitude in zip(airports['IATA'], airports['Latitude'])}

In [87]:
flights_grouped['start_long'] = flights_grouped['origin'].replace(airports_long)
flights_grouped['start_lati'] = flights_grouped['origin'].replace(airports_lati)
flights_grouped['end_long'] = flights_grouped['dest'].replace(airports_long)
flights_grouped['end_lati'] = flights_grouped['dest'].replace(airports_lati)
flights_grouped = flights_grouped[~flights_grouped['end_long'].isin(['ANC','BQN', 'HNL', 'PSE', 'SJU', 'STT'])]
flights_grouped.head()

origin dest                   carrier arr_delay                          \
                                            count       mean   min    max   
0    EWR  ALB  ExpressJet Airlines Inc.       418  14.397129 -34.0  328.0   
2    EWR  ATL         Endeavor Air Inc.         4  -6.250000 -24.0   15.0   
3    EWR  ATL      Delta Air Lines Inc.      3116   9.996470 -38.0  796.0   
4    EWR  ATL  ExpressJet Airlines Inc.      1654  19.546554 -39.0  454.0   
5    EWR  ATL     United Air Lines Inc.       102  10.500000 -37.0  360.0   

  air_time                      ...    distance      air_speed (mph)  \
     count        mean    min   ...         min  max           count   
0      418   31.787081   24.0   ...         143  143             418   
2        4  111.000000  105.0   ...         746  746               4   
3     3116  110.853659   88.0   ...         746  746            3116   
4     1654  114.042322   90.0   ...         746  746            1654   
5      102  113.647059   97.0   ...         746  746             102   

                                      start_long start_lati end_long end_lati  
         mean         min         max                                          
0  272.300976  171.600000  357.500000 -74.168701  40.692501 -73.8017  42.7483  
2  403.899553  382.564103  426.285714 -74.168701  40.692501 -84.4281  33.6367  
3  406.542093  254.318182  508.636364 -74.168701  40.692501 -84.4281  33.6367  
4  395.945542  278.012422  497.333333 -74.168701  40.692501 -84.4281  33.6367  
5  396.577226  315.211268  461.443299 -74.168701  40.692501 -84.4281  33.6367  

[5 rows x 23 columns]

In [89]:
from bokeh.sampledata.us_states import data as states

if 'HI' in states: del states['HI']
if 'AK' in states: del states['AK']

In [90]:
xs = [states[state]['lons'] for state in states]
ys = [states[state]['lats'] for state in states]

p = figure(plot_width=1200, plot_height=760, title = 'United States')
p.xaxis.visible=False
p.yaxis.visible=False
p.grid.visible=False
p.patches(xs, ys, fill_alpha = 0.0, line_color = 'gray')

show(p)

In [91]:
x = [[start_long, end_long] for start_long, end_long in zip(flights_grouped['start_long'], flights_grouped['end_long'])]
y = [[start_lati, end_lati] for start_lati, end_lati in zip(flights_grouped['start_lati'], flights_grouped['end_lati'])]

In [92]:
p.multi_line(x, y, color = 'red')
show(p)

In [93]:
flights_grouped.to_csv('data/flights_map.csv')